# Домашнее задание № 3. Исправление опечаток

In [2]:
!pip install textdistance

  Obtaining dependency information for textdistance from https://files.pythonhosted.org/packages/44/7b/13e25cbe91947672aeec2637b227398c3e97ec04ad0eaaac9125726ab0e7/textdistance-4.6.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
import textdistance
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from tqdm.notebook import tqdm
from collections import defaultdict

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [2]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [3]:
# создаем словарь
vocab = Counter(re.findall('\w+', corpus.lower()))

In [149]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [150]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [80]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [81]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])    

    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [8]:
get_closest_match_vec('сонце', X, vec)

[('монце', 0.24999999999999978),
 ('донце', 0.24999999999999978),
 ('конце', 0.24999999999999978),
 ('херсонцев', 0.2640199278060129),
 ('саксонцев', 0.2640199278060129),
 ('сон', 0.2928932188134523),
 ('олонце', 0.2998599579859951),
 ('ньонце', 0.2998599579859952),
 ('соне', 0.3264246859454365),
 ('донцем', 0.3291796067500631),
 ('монцей', 0.3291796067500631),
 ('солнце', 0.3291796067500631),
 ('концессионное', 0.33217692887937167),
 ('концерн', 0.3545027756320972),
 ('монцезе', 0.3545027756320972),
 ('концессионном', 0.35948738477965136),
 ('концессионера', 0.36155760193093855),
 ('концессионером', 0.36555873142548445),
 ('лондонцев', 0.367544467966324),
 ('эстонцев', 0.3700592116512881)]

In [31]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]

    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    res = max(closest, key = P)

    return res


def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1



In [10]:
get_closest_hybrid_match('легушька', X, vec)

('лягушка', 0.75)

In [11]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [12]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7928964482241121
0.0015527950310559005
0.09004249454461927


## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [3]:
vocab = dict(Counter(re.findall('\w+', corpus.lower())))

In [184]:
def generate_deletion_dict(words):
    deletion_dict = dict()

    for word in words:
        for deletion_variant in generate_deletions(word):
            deletion_dict[deletion_variant] = word
    return deletion_dict

def generate_deletions(word):
    deletions = set()
    for i in range(len(word) + 1):
        deletion = word[:i] + word[i+1:]
        deletions.add(deletion)
    return deletions

def corrected(typo, deletion_dict, words):
 
    #Генерируются все варианты удаления для опечатки
    deletion_variants = generate_deletions(typo)

    possible_corrections = []
    for variant in deletion_variants:
        possible_corrections.append(deletion_dict.get(variant, []))

    #Проверка, если слово не исправляется, просто не меняем его
    all_empty = all(not sublist for sublist in possible_corrections)
    if all_empty:
        return typo
    
    #Находим самое вероятное слово
    scores = dict()
    for correction in possible_corrections:
        if words.get(str(correction)) != None:
            scores[correction] = words.get(correction,[])
    
    best_correction = max(scores, key=scores.get)
   
    return best_correction

In [73]:
part_of_vocab = dict(list(vocab.items())[0: 10]) 
part_of_vocab

{'новостройка': 10,
 'нижегородская': 32,
 'область': 3387,
 'новостро': 1,
 'йка': 6,
 'сельский': 451,
 'посёлок': 818,
 'в': 267296,
 'дивеевском': 9,
 'районе': 3790}

In [42]:
generate_deletions('пуховик')

{'пуовик',
 'пухвик',
 'пухови',
 'пуховик',
 'пуховк',
 'пухоик',
 'пховик',
 'уховик'}

In [174]:
vocab['пиво']

17

In [186]:
correct_words = vocab
deletion_dict = generate_deletion_dict(vocab)

typo_word = "яблко"
corrected_word = corrected(typo_word, deletion_dict, correct_words)
print(f"Original word: {typo_word}")
print(f"Corrected word: {corrected_word}")

Original word: яблко
Corrected word: яблоко


In [188]:
deletion_dict = generate_deletion_dict(vocab)

In [189]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], corrected(pair[1], deletion_dict, vocab))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [ ]:
#Отработал гораздо быстрее!

In [190]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.46673336668334164
0.3781055900621118
0.520156196164006
